In [1]:
import pennylane as qml
from pennylane import numpy as np
from embeddings_circuit import generate_data, generate_grid_data, embedding_circuit
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='notebook', font='serif')

In [2]:
CSWAP = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                  [0, 1, 0, 0, 0, 0, 0, 0],
                  [0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 1, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0],
                  [0, 0, 0, 0, 0, 0, 1, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0],
                  [0, 0, 0, 0, 0, 0, 0, 1]])

In [3]:
def featmap(x,weights,wires):
    """Wrapper for feature map to define specific keyword arguments."""
    return embedding_circuit(x,weights,wires)

In [4]:
margin = 0.1   # choose 0.1 
data_size = 40   # chose and 40 for random data
n_layers = 1  # number of layers for featuremap, if applicable
n_inp = 1  # number of wires that feature map acts on
n_all = 2*n_inp + 1

pennylane_dev = 'default.qubit'
dev = qml.device(pennylane_dev, wires=n_all)   # Device to |<x|x'>|^2 using the SWAP trick.

In [5]:
np.random.seed(137)
X, Y = generate_data(data_size,margin)

In [6]:
# Divide inputs into classes
A = []
B = []
for i in range(len(Y)):
    if Y[i] == 1:
        B.append(X[i])
    else:
        A.append(X[i])

In [7]:
@qml.qnode(dev, cache=True)
def circuit(weights, x1=None, x2=None):

    # Load the two inputs into two different registers
    featmap(x1,weights, range(1, n_inp+1))
    featmap(x2,weights, range(n_inp+1, 2*n_inp+1))

    # Do a SWAP test
    qml.Hadamard(wires=0)
    for k in range(n_inp):
        qml.QubitUnitary(CSWAP, wires=[0, k+1, n_inp+k+1])
    qml.Hadamard(wires=0)

    # Measure overlap by checking ancilla
    return qml.expval(qml.PauliZ(0))

def tr_rr(weights, A=None):
    # Compute intra-class overlap A
    tr_rr = 0
    for a1 in A:
        for a2 in A:
            tr_rr += circuit(weights, x1=a1, x2=a2)
    tr_rr = tr_rr / len(A)**2
    return tr_rr

def tr_ss(weights, B=None):
    # Compute intra-class overlap B
    tr_ss = 0
    for b1 in B:
        for b2 in B:
            tr_ss += circuit(weights, x1=b1, x2=b2)
    tr_ss = tr_ss/len(B)**2
    return tr_ss

def tr_rs(weights, A=None, B=None):
    # Compute inter-class overlap A-B
    tr_rs = 0
    for a in A:
        for b in B:
            tr_rs += circuit(weights, x1=a, x2=b)
    tr_rs = tr_rs/(len(A)*len(B))
    return tr_rs

def cost(weights, A=None, B=None):

    # Fidelity cost,
    rr = tr_rr(weights, A=A)
    ss = tr_ss(weights, B=B)
    rs = tr_rs(weights, A=A, B=B)
    distance = - rs + 0.5 * (ss + rr)
    return 1 - distance  # min is 0

In [15]:
def classifier(x_new,weights, A=None, B=None):
    
    overlap_A = 0
    for a in A:
        overlap_A += circuit(weights, x1=x_new, x2=a)
    overlap_A = overlap_A/len(A)
    
    overlap_B = 0
    for b in B:
        overlap_B += circuit(weights, x1=x_new, x2=b)
    overlap_B = overlap_B/len(B)
    
    #return np.tanh(len(A)*(overlap_B-overlap_A)) 
    
    if overlap_A > overlap_B:
        return -1
    elif overlap_A < overlap_B:
        return 1
    else:
        return 0

def risk(weights,A=None,B=None):
    
    dataset_size = len(A)+len(B)
    I = 0
    
    for a in A:
        I = I - classifier(a,weights,A,B)
    for b in B:
        I = I + classifier(b,weights,A,B)
    
    I = I/dataset_size
    
    return 0.5 - 0.5*I


# We now define 'smooth' and 'differentiable' version
# of risk function. We do this by replacing 
# sign(fidelity) with tanh(fidelity).

def smooth_classifier(x_new,weights, A=None, B=None):
    
    overlap_A = 0
    for a in A:
        overlap_A += circuit(weights, x1=x_new, x2=a)
    overlap_A = overlap_A/len(A)
    
    overlap_B = 0
    for b in B:
        overlap_B += circuit(weights, x1=x_new, x2=b)
    overlap_B = overlap_B/len(B)
    
    return np.tanh(len(A)*(overlap_B-overlap_A))

def smooth_risk(weights,A=None,B=None):
    
    dataset_size = len(A)+len(B)
    I = 0
    
    for a in A:
        I = I - smooth_classifier(a,weights,A,B)
    for b in B:
        I = I + smooth_classifier(b,weights,A,B)
    
    I = I/dataset_size
    
    return 0.5 - 0.5*I

In [9]:
th = np.linspace(0, 2.0*np.pi, 50)

cost_th = np.array([cost([[th_]], A=A, B=B) for th_ in th])
risk_th = np.array([risk([[th_]], A=A, B=B) for th_ in th])

In [ ]:
fig = plt.figure(figsize=(12, 6))
# Plotting 1: original data
ax1 = fig.add_subplot(2, 3, 1)
ax1.set_title("Data set 1", pad=20)
A = np.array(A)
B = np.array(B)
ax1.scatter(A[:, 0], A[:, 1], c='r')
ax1.scatter(B[:, 0], B[:, 1], c='b')
ax1.set_ylim((-0.1-0.5*np.pi, 0.1+0.5*np.pi))
ax1.set_xlim((-0.1-0.5*np.pi, 0.1+0.5*np.pi))

# Plotting the HS cost
ax2 = fig.add_subplot(2, 3, 2)
ax2.set_title("HS cost function", pad=20)
ax2.plot(th, cost_th,color='green', marker='', linestyle='-', linewidth=2.5)
ax2.set_ylim((-0.1, 1.1))
ax2.set_xlim((0, 2.0*np.pi))
ax2.set_xlabel("theta")

# Plotting the risk function
ax3 = fig.add_subplot(2, 3, 3)
ax3.set_title("Risk function", pad=20)
ax3.plot(th, risk_th,color='green', marker='', linestyle='-', linewidth=2.5)
ax3.set_ylim((-0.1, 1.1))
ax3.set_xlim((0, 2.0*np.pi))
ax3.set_xlabel("theta")